# 📊 Evaluation Loops - Avaliação de Modelos Fine-Tunados

Este notebook implementa **evaluation loops profissionais** para avaliar modelos fine-tunados.

## 🎯 O que este notebook faz:

1. ✅ **Carrega modelos** (base vs fine-tuned)
2. ✅ **Métricas automáticas** (ROUGE, BLEU, BERTScore, Perplexity)
3. ✅ **Comparação lado a lado** (qual modelo responde melhor?)
4. ✅ **Benchmark de domínio** (perguntas de Retail Media)
5. ✅ **Relatórios e visualizações**
6. ✅ **Integração com MLflow** (tracking de experimentos)

## 📋 Pré-requisitos:

- ✅ Modelo fine-tunado salvo (do notebook `fine_tuning_qlora_colab.ipynb`)
- ✅ Dataset de teste com respostas de referência
- ✅ GPU recomendada (pode rodar em CPU mas é lento)

## 🏢 Boas Práticas:

Este notebook segue padrões de empresas como OpenAI, Google, Anthropic:
- ✅ Separação treino/avaliação
- ✅ Métricas objetivas e reproduzíveis
- ✅ Tracking de experimentos
- ✅ Comparação sistemática

---


## 📦 Instalar Dependências


In [ ]:
# Instalar dependências (se necessário)
%pip install -q transformers accelerate peft bitsandbytes
%pip install -q rouge-score bert-score nltk sacrebleu
%pip install -q pandas matplotlib seaborn
%pip install -q mlflow

print("✅ Dependências instaladas!")


## ⚙️ Imports e Configuração


In [ ]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from typing import Dict, List
import json
from datetime import datetime

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
from rouge_score import rouge_scorer
from bert_score import BERTScorer
from sacrebleu.metrics import BLEU
import mlflow

import nltk
nltk.download('punkt', quiet=True)

print("✅ Bibliotecas importadas!")
print(f"🔥 PyTorch: {torch.__version__}")
print(f"🤗 Transformers: {__import__('transformers').__version__}")


## 🔧 Configurações - **AJUSTE AQUI**


In [ ]:
# ========================================
# CONFIGURAÇÕES - AJUSTE AQUI
# ========================================

# Modelo base (mesmo usado no fine-tuning)
BASE_MODEL_NAME = "microsoft/phi-2"

# Caminho do modelo fine-tunado (adaptadores LoRA)
FINETUNED_MODEL_PATH = "./models/phi2-retail-media"
# No Colab: "/content/drive/MyDrive/finetuned_models/phi2-retail-media"

# Dataset de teste com respostas de referência
TEST_DATASET_PATH = "./data/evaluation/test_questions.json"

# Diretório de saída
OUTPUT_DIR = "./evaluation_results"
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

# MLflow
EXPERIMENT_NAME = "model-evaluation"

# Parâmetros de geração
MAX_NEW_TOKENS = 512
TEMPERATURE = 0.7
TOP_P = 0.9

print("✅ Configuração definida!")
print(f"📦 Modelo base: {BASE_MODEL_NAME}")
print(f"🎯 Modelo fine-tuned: {FINETUNED_MODEL_PATH}")


## 📚 Carregar Classe ModelEvaluator

Usaremos a classe reutilizável do módulo `src/evaluation/`


In [ ]:
# Adicionar src ao path
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent))

from src.evaluation.evaluator import ModelEvaluator, compare_models

print("✅ ModelEvaluator carregado!")


## 📂 Criar Dataset de Teste

Formato esperado:
```json
[
  {
    "question": "O que é ACOS?",
    "reference_answer": "ACOS (Advertising Cost of Sale) é...",
    "category": "metricas"  // opcional
  }
]
```


In [ ]:
# Criar dataset de exemplo se não existir
if not Path(TEST_DATASET_PATH).exists():
    print("⚠️ Dataset não encontrado. Criando exemplo...")
    
    example_data = [
        {
            "question": "O que é Retail Media?",
            "reference_answer": "Retail Media é uma forma de publicidade digital onde varejistas monetizam seus sites e aplicativos vendendo espaços publicitários para marcas e fabricantes.",
            "category": "conceitos"
        },
        {
            "question": "Explique o que é ACOS",
            "reference_answer": "ACOS (Advertising Cost of Sale) é uma métrica que indica quanto você gasta em publicidade para cada real de venda gerada. Calcula-se dividindo o custo da publicidade pela receita gerada.",
            "category": "metricas"
        },
        {
            "question": "O que é RTB?",
            "reference_answer": "RTB (Real-Time Bidding) é um processo de compra automatizada de espaços publicitários em tempo real através de leilões.",
            "category": "conceitos"
        },
    ]
    
    Path(TEST_DATASET_PATH).parent.mkdir(parents=True, exist_ok=True)
    with open(TEST_DATASET_PATH, 'w', encoding='utf-8') as f:
        json.dump(example_data, f, ensure_ascii=False, indent=2)
    
    print(f"✅ Dataset exemplo criado: {TEST_DATASET_PATH}")
    print("💡 Substitua por seu dataset real!")

# Carregar dataset
with open(TEST_DATASET_PATH, 'r', encoding='utf-8') as f:
    test_data = json.load(f)

print(f"\n📊 Dataset carregado: {len(test_data)} exemplos")


## 🤖 Carregar Modelos (Base + Fine-Tuned)


In [ ]:
print("🔄 Carregando modelos...\\n")

# Configurar quantização 4-bit
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# 1. Carregar modelo BASE
print("🔵 Carregando modelo BASE...")
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

# Carregar tokenizer
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print(f"✅ Modelo base carregado: {BASE_MODEL_NAME}")

# 2. Carregar modelo FINE-TUNED
print("\\n🟢 Carregando modelo FINE-TUNED...")
finetuned_model = PeftModel.from_pretrained(
    base_model,
    FINETUNED_MODEL_PATH
)

print(f"✅ Modelo fine-tuned carregado: {FINETUNED_MODEL_PATH}")
print("\\n✅ Ambos os modelos prontos para avaliação!")


## 📊 Criar Evaluators


In [ ]:
print("🔄 Criando evaluators...\\n")

# Evaluator para modelo base
base_evaluator = ModelEvaluator(
    model=base_model,
    tokenizer=tokenizer,
    model_name="base",
    prompt_format="alpaca"
)

# Evaluator para modelo fine-tuned  
finetuned_evaluator = ModelEvaluator(
    model=finetuned_model,
    tokenizer=tokenizer,
    model_name="finetuned",
    prompt_format="alpaca"
)

print("\\n✅ Evaluators prontos para uso!")


## 🚀 EVALUATION LOOP - Avaliar Dataset Completo


In [ ]:
print("="*80)
print("🚀 INICIANDO EVALUATION LOOP")
print("="*80)

# Avaliar modelo base
print("\\n🔵 Avaliando modelo BASE...")
base_results = base_evaluator.evaluate_dataset(
    test_data,
    verbose=True,
    save_path=f"{OUTPUT_DIR}/base_results.csv"
)

# Avaliar modelo fine-tuned
print("\\n🟢 Avaliando modelo FINE-TUNED...")
finetuned_results = finetuned_evaluator.evaluate_dataset(
    test_data,
    verbose=True,
    save_path=f"{OUTPUT_DIR}/finetuned_results.csv"
)

print("\\n" + "="*80)
print("✅ EVALUATION LOOP CONCLUÍDO!")
print("="*80)


## 📈 Comparação de Resultados


In [ ]:
# Combinar resultados
all_results = pd.concat([base_results, finetuned_results], ignore_index=True)

# Métricas principais
metric_columns = ['rouge1_f', 'rouge2_f', 'rougeL_f', 'bleu', 'bertscore_f1']

# Calcular médias por modelo
comparison = all_results.groupby('model')[metric_columns].agg(['mean', 'std'])

print("\\n📊 COMPARAÇÃO DE MÉTRICAS")
print("="*80)
print(comparison.round(4))

# Calcular melhorias
print("\\n📈 MELHORIA DO FINE-TUNING:")
print("="*80)
for metric in metric_columns:
    base_score = base_results[metric].mean()
    finetuned_score = finetuned_results[metric].mean()
    improvement = ((finetuned_score - base_score) / base_score) * 100
    symbol = "✅" if improvement > 0 else "⚠️"
    print(f"{symbol} {metric:20s}: {improvement:+.2f}%")


## 📊 Visualizações


In [ ]:
# Gráfico de comparação
plt.figure(figsize=(12, 6))

means = all_results.groupby('model')[metric_columns].mean()
means.T.plot(kind='bar', figsize=(12, 6), rot=45)

plt.title('Comparação de Métricas: Base vs Fine-Tuned', fontsize=14, fontweight='bold')
plt.xlabel('Métrica', fontsize=11)
plt.ylabel('Score', fontsize=11)
plt.legend(['Modelo Base', 'Modelo Fine-Tuned'])
plt.ylim(0, 1.0)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.savefig(f"{OUTPUT_DIR}/metrics_comparison.png", dpi=300)
plt.show()

print(f"✅ Gráfico salvo: {OUTPUT_DIR}/metrics_comparison.png")


## 🔬 Integração com MLflow


In [ ]:
# Configurar MLflow
mlflow.set_experiment(EXPERIMENT_NAME)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Log resultados
with mlflow.start_run(run_name=f"eval_{timestamp}"):
    # Parâmetros
    mlflow.log_param("base_model", BASE_MODEL_NAME)
    mlflow.log_param("finetuned_model", FINETUNED_MODEL_PATH)
    mlflow.log_param("test_samples", len(test_data))
    
    # Métricas do modelo base
    for metric in metric_columns:
        mlflow.log_metric(f"base_{metric}", base_results[metric].mean())
    
    # Métricas do modelo fine-tuned
    for metric in metric_columns:
        mlflow.log_metric(f"finetuned_{metric}", finetuned_results[metric].mean())
    
    # Melhorias
    for metric in metric_columns:
        improvement = ((finetuned_results[metric].mean() - base_results[metric].mean()) / base_results[metric].mean()) * 100
        mlflow.log_metric(f"improvement_{metric}", improvement)
    
    # Artifacts
    mlflow.log_artifact(f"{OUTPUT_DIR}/base_results.csv")
    mlflow.log_artifact(f"{OUTPUT_DIR}/finetuned_results.csv")
    mlflow.log_artifact(f"{OUTPUT_DIR}/metrics_comparison.png")

print("\\n✅ Resultados logados no MLflow!")
print("🔬 Para visualizar: mlflow ui --port 5000")


## 📋 Resumo Final


In [ ]:
print("\\n" + "="*80)
print("📋 RESUMO DA AVALIAÇÃO")
print("="*80)

avg_improvement = sum([
    ((finetuned_results[m].mean() - base_results[m].mean()) / base_results[m].mean()) * 100
    for m in metric_columns
]) / len(metric_columns)

print(f"\\n🎯 Melhoria Média Geral: {avg_improvement:+.2f}%")

if avg_improvement > 5:
    print("\\n✅ CONCLUSÃO: Fine-tuning foi EFETIVO! 🎉")
    print("   O modelo fine-tuned tem performance significativamente melhor.")
elif avg_improvement > 0:
    print("\\n⚠️ CONCLUSÃO: Fine-tuning teve melhoria LEVE.")
    print("   Considere: mais dados de treino, mais epochs, ou ajustar hyperparâmetros.")
else:
    print("\\n❌ CONCLUSÃO: Fine-tuning não melhorou.")
    print("   Revisar: qualidade do dataset, formato dos dados, hyperparâmetros.")

print(f"\\n📁 Arquivos gerados em: {OUTPUT_DIR}/")
print(f"   - base_results.csv")
print(f"   - finetuned_results.csv")
print(f"   - metrics_comparison.png")

print("\\n" + "="*80)
print("✅ AVALIAÇÃO COMPLETA CONCLUÍDA!")
print("="*80)
